### Imports

In [4]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

from models.mnist_resnet import resnet50
from models.mnist_resnet import resnet18


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

# Model settings

In [2]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 128
NUM_EPOCHS = 20

# Architecture
NUM_FEATURES = 28*28
NUM_CLASSES = 10

# Other
DEVICE = "cuda:0"
GRAYSCALE = True

device = torch.device(DEVICE)

## MNIST Dataset

In [3]:
# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


## Cargar el Modelo Resnet50

In [5]:
torch.manual_seed(RANDOM_SEED)

model = resnet18(NUM_CLASSES)
model.to(DEVICE)
 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

### Fun Accuracy

In [6]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        if(targets.size() == torch.Size([128])):
            features = features.to(device)
            targets = targets.to(device)

            logits, probas = model(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    print("correctos: ", correct_pred ,'| total: ',num_examples )
    return correct_pred.float() / num_examples * 100

# Entrenamiento

In [7]:
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/020 | Batch 0000/0469 | Cost: 2.6381
Epoch: 001/020 | Batch 0050/0469 | Cost: 0.2107
Epoch: 001/020 | Batch 0100/0469 | Cost: 0.1657
Epoch: 001/020 | Batch 0150/0469 | Cost: 0.0663
Epoch: 001/020 | Batch 0200/0469 | Cost: 0.0893
Epoch: 001/020 | Batch 0250/0469 | Cost: 0.0987
Epoch: 001/020 | Batch 0300/0469 | Cost: 0.0420
Epoch: 001/020 | Batch 0350/0469 | Cost: 0.0510
Epoch: 001/020 | Batch 0400/0469 | Cost: 0.0663
Epoch: 001/020 | Batch 0450/0469 | Cost: 0.0265
correctos:  tensor(59218, device='cuda:0') | total:  59904
Epoch: 001/020 | Train: 98.855%
Time elapsed: 0.19 min
Epoch: 002/020 | Batch 0000/0469 | Cost: 0.1462
Epoch: 002/020 | Batch 0050/0469 | Cost: 0.0143
Epoch: 002/020 | Batch 0100/0469 | Cost: 0.0236
Epoch: 002/020 | Batch 0150/0469 | Cost: 0.0459
Epoch: 002/020 | Batch 0200/0469 | Cost: 0.0559
Epoch: 002/020 | Batch 0250/0469 | Cost: 0.0375
Epoch: 002/020 | Batch 0300/0469 | Cost: 0.0224
Epoch: 002/020 | Batch 0350/0469 | Cost: 0.0239
Epoch: 002/020 | Batch

## Guardar el modelo entrenado

In [8]:
torch.save(model.state_dict(), 'weights/mnist18.pth')

### Evaluar el modelo entrenado

In [9]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))


correctos:  tensor(9880, device='cuda:0') | total:  9984
Test accuracy: 98.96%


## Cargar pesos y evaluar dnvo

In [11]:
# Crear una instancia del modelo con la misma arquitectura
model = resnet18(NUM_CLASSES)
model.to(DEVICE)

# Cargar los pesos del modelo
model.load_state_dict(torch.load('weights/mnist18.pth'))

# Cambiar el modelo a eval() para usarlo en inferencia
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))


correctos:  tensor(9888, device='cuda:0') | total:  9984
Test accuracy: 99.04%
